### 涨跌停数据库建立

#### 导入相关模块

In [1]:
import tushare as ts
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import datetime as dt
import time 
from pyecharts.charts import Bar,Line,Scatter,Pie
from pyecharts import options as opts
from pyecharts.render import make_snapshot
from snapshot_phantomjs import snapshot
from pyecharts.globals import ThemeType

#### 设置输出显示
##### 单元格设置
+ 允许一个单元格有多个输出
+ 显示所有列
+ 不换行

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # 允许一个单元格有多个输出
pd.set_option('display.max_columns', None)  # 显示所有列，None无限制
pd.set_option('expand_frame_repr', False)  # 不换行

##### 设置中英文列名对照
+ 按tushare中对应函数的输出参数顺序

In [19]:
llCname = {
    'pct_chg': '涨跌幅',
    'amp': '振幅',
    'fc_ratio': '封单额/日成交额',
    'fl_ratio': '封单手数/流通股本',
    'fd_amount': '封单额',
    'first_time': '首次涨停',
    'last_time': '最后封板时',
    'open_times': '打开次数',
    'strth': '涨跌停强度',
    'limit': 'D跌停U涨停',
}

#### 设置数据目录

In [4]:
Stock_Data_Dir=Path.home()/'Stock_Data/Tushare'
if (not Stock_Data_Dir.exists()):
    Stock_Data_Dir.mkdir()
out_csv_dir=Stock_Data_Dir/'csv'
if(not out_csv_dir.exists()):  #Path.exists()
    out_csv_dir.mkdir()

#### 设置token并初始化接口¶

In [5]:
tokenfile=Stock_Data_Dir/'Token'/'token.pk'
with open(tokenfile,'rb') as f :
    mytoken=pickle.load(f)
ts.set_token(mytoken)   
pro = ts.pro_api()

#### 设置查询时间

In [15]:
sdate,edate='20200301','20200327'

### 获取数据

#### 查询当前所有正常上市交易的股票列表

In [7]:
Slist = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,name')
#Slist.head(3)
#Slist.info()

,ts_code,name
0,000001.SZ,平安银行
1,000002.SZ,万科A
2,000004.SZ,国农科技


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3800 entries, 0 to 3799
Data columns (total 2 columns):
ts_code    3800 non-null object
name       3800 non-null object
dtypes: object(2)
memory usage: 59.5+ KB


#### 指定时间段的交易日历

In [11]:
trade_cal = pro.trade_cal(exchange='',
                          start_date=sdate,
                          end_date=edate,
                          is_open=1)['cal_date']
trade_cal

0     20200302
1     20200303
2     20200304
3     20200305
4     20200306
5     20200309
6     20200310
7     20200311
8     20200312
9     20200313
10    20200316
11    20200317
12    20200318
13    20200319
14    20200320
15    20200323
16    20200324
17    20200325
18    20200326
19    20200327
Name: cal_date, dtype: object

#### 涨跌停数据汇总
**有两种方法获取数据**
+ 按交易日历遍历日期，可获得指定时间内所有所有涨跌停股票的数据
+ 按遍历股票，筛出在指定时间段内有过涨跌停的记录

##### 遍历日期获取每个交易日所有涨跌停股票数据
+ 获取的数据中含有在此时间段退市的股票
+ tushare单日最大可获取涨跌股票数量有变化-约3千多条

In [12]:
limit_data=[]
for date in trade_cal:
    while(True):
        try :
            df=pro.limit_list(trade_date=date) #try 语句块中异常发生点后的剩余语句永远不会到达(所以就不会执行)。
            limit_data.append(df)   #先将表构成list，然后在作为concat的输入进行合并，这样效率更高
            print('正在处理{}'.format(date))
            break
        except:
            print('因超时等待10秒重试......')
            time.sleep(10)   
    
date_result=pd.concat(limit_data,ignore_index=True)
print('OK!')

正在处理20200302
正在处理20200303
正在处理20200304
正在处理20200305
正在处理20200306
正在处理20200309
正在处理20200310
正在处理20200311
正在处理20200312
正在处理20200313
正在处理20200316
正在处理20200317
正在处理20200318
正在处理20200319
正在处理20200320
正在处理20200323
正在处理20200324
正在处理20200325
正在处理20200326
正在处理20200327
OK!


In [13]:
# 剔除在此期间退市的股票
eff_code=pd.merge(Slist['ts_code'],date_result,on='ts_code',how='inner')
eff_code.head()
eff_code.info()

,ts_code,trade_date,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,000004.SZ,20200302,国农科技,44.77,10.00,5.26,16.4257,1.3949,51583548.0,09:37:48,09:37:48,0,66.452320,U
1,000016.SZ,20200305,深康佳Ａ,11.01,9.99,13.79,5.1189,0.9882,173715824.0,14:04:27,14:04:27,0,51.771300,U
2,000016.SZ,20200310,深康佳Ａ,12.38,10.04,10.58,2.4061,0.5504,108784896.0,13:11:36,14:00:15,14,49.688126,U
3,000017.SZ,20200303,深中华A,4.21,9.92,10.70,66.5446,2.2464,28653682.0,14:54:21,14:54:21,0,61.066780,U
4,000017.SZ,20200313,深中华A,5.15,10.04,14.74,9.6176,1.0748,16770450.0,09:56:27,10:03:21,1,46.275497,U


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2494 entries, 0 to 2493
Data columns (total 14 columns):
ts_code       2494 non-null object
trade_date    2494 non-null object
name          2494 non-null object
close         2494 non-null float64
pct_chg       2494 non-null float64
amp           2494 non-null float64
fc_ratio      2494 non-null float64
fl_ratio      2494 non-null float64
fd_amount     2494 non-null float64
first_time    2494 non-null object
last_time     2494 non-null object
open_times    2494 non-null int64
strth         2494 non-null float64
limit         2494 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 292.3+ KB


##### 遍历个股在指定时间内的涨跌停情况

In [11]:
#df.empty ，这是 DataFrame 内置的属性，可以看到虽然调用简单，但他是最耗时的
#len(df)==0 ，这是通过Python内置len方法判断 DataFrame 的行数，相对来说速度比较快，是第1种的3倍
#len(df.index)==0 ，这是判断 DataFrame 的行索引的值数量，这已经到达纳秒级别了，是其中最快的方式
limit_data=[]
limit_stcok=[]     #在统计时间内有哪些股票涨跌停过

for code in Slist.ts_code:
    while(True):
        try :
            df=pro.limit_list(ts_code=code,start_date=sdate, end_date=edate)
            print('正在处理{}'.format(code))
            break
        except :
            print('因超时等待10秒重试......')
            time.sleep(10)    
    if(len(df.index)):
        limit_data.append(df)
        limit_stcok.append(code)

stock_result=pd.concat(limit_data,ignore_index=True) 
print('在{}至{}期间，共有{}只股票涨跌停'.format(sdate,edate,len(set(limit_stcok))))      

正在处理000001.SZ
正在处理000002.SZ
正在处理000004.SZ
正在处理000005.SZ
正在处理000006.SZ
正在处理000007.SZ
正在处理000008.SZ
正在处理000009.SZ
正在处理000010.SZ
正在处理000011.SZ
正在处理000012.SZ
正在处理000014.SZ
正在处理000016.SZ
正在处理000017.SZ
正在处理000019.SZ
正在处理000020.SZ
正在处理000021.SZ
正在处理000023.SZ
正在处理000025.SZ
正在处理000026.SZ
正在处理000027.SZ
正在处理000028.SZ
正在处理000029.SZ
正在处理000030.SZ
正在处理000031.SZ
正在处理000032.SZ
正在处理000034.SZ
正在处理000035.SZ
正在处理000036.SZ
正在处理000037.SZ
正在处理000038.SZ
正在处理000039.SZ
正在处理000040.SZ
正在处理000042.SZ
正在处理000045.SZ
正在处理000046.SZ
正在处理000048.SZ
正在处理000049.SZ
正在处理000050.SZ
正在处理000055.SZ
正在处理000056.SZ
正在处理000058.SZ
正在处理000059.SZ
正在处理000060.SZ
正在处理000061.SZ
正在处理000062.SZ
正在处理000063.SZ
正在处理000065.SZ
正在处理000066.SZ
正在处理000068.SZ
正在处理000069.SZ
正在处理000070.SZ
正在处理000078.SZ
正在处理000088.SZ
正在处理000089.SZ
正在处理000090.SZ
正在处理000096.SZ
正在处理000099.SZ
正在处理000100.SZ
正在处理000150.SZ
正在处理000151.SZ
正在处理000153.SZ
正在处理000155.SZ
正在处理000156.SZ
正在处理000157.SZ
正在处理000158.SZ
正在处理000159.SZ
正在处理000166.SZ
正在处理000301.SZ
正在处理000333.SZ
正在处理000338.SZ
正在处理00

正在处理002129.SZ
正在处理002130.SZ
正在处理002131.SZ
正在处理002132.SZ
正在处理002133.SZ
正在处理002134.SZ
正在处理002135.SZ
正在处理002136.SZ
正在处理002137.SZ
正在处理002138.SZ
正在处理002139.SZ
正在处理002140.SZ
正在处理002141.SZ
正在处理002142.SZ
正在处理002144.SZ
正在处理002145.SZ
正在处理002146.SZ
正在处理002147.SZ
正在处理002148.SZ
正在处理002149.SZ
正在处理002150.SZ
正在处理002151.SZ
正在处理002152.SZ
正在处理002153.SZ
正在处理002154.SZ
正在处理002155.SZ
正在处理002156.SZ
正在处理002157.SZ
正在处理002158.SZ
正在处理002159.SZ
正在处理002160.SZ
正在处理002161.SZ
正在处理002162.SZ
正在处理002163.SZ
正在处理002164.SZ
正在处理002165.SZ
正在处理002166.SZ
正在处理002167.SZ
正在处理002168.SZ
正在处理002169.SZ
正在处理002170.SZ
正在处理002171.SZ
正在处理002172.SZ
正在处理002173.SZ
正在处理002174.SZ
正在处理002175.SZ
正在处理002176.SZ
正在处理002177.SZ
正在处理002178.SZ
正在处理002179.SZ
正在处理002180.SZ
正在处理002181.SZ
正在处理002182.SZ
正在处理002183.SZ
正在处理002184.SZ
正在处理002185.SZ
正在处理002186.SZ
正在处理002187.SZ
正在处理002188.SZ
正在处理002189.SZ
正在处理002190.SZ
正在处理002191.SZ
正在处理002192.SZ
正在处理002193.SZ
正在处理002194.SZ
正在处理002195.SZ
正在处理002196.SZ
正在处理002197.SZ
正在处理002198.SZ
正在处理002199.SZ
正在处理002200.SZ
正在处理00

正在处理002725.SZ
正在处理002726.SZ
正在处理002727.SZ
正在处理002728.SZ
正在处理002729.SZ
正在处理002730.SZ
正在处理002731.SZ
正在处理002732.SZ
正在处理002733.SZ
正在处理002734.SZ
正在处理002735.SZ
正在处理002736.SZ
正在处理002737.SZ
正在处理002738.SZ
正在处理002739.SZ
正在处理002740.SZ
正在处理002741.SZ
正在处理002742.SZ
正在处理002743.SZ
正在处理002745.SZ
正在处理002746.SZ
正在处理002747.SZ
正在处理002748.SZ
正在处理002749.SZ
正在处理002750.SZ
正在处理002751.SZ
正在处理002752.SZ
正在处理002753.SZ
正在处理002755.SZ
正在处理002756.SZ
正在处理002757.SZ
正在处理002758.SZ
正在处理002759.SZ
正在处理002760.SZ
正在处理002761.SZ
正在处理002762.SZ
正在处理002763.SZ
正在处理002765.SZ
正在处理002766.SZ
正在处理002767.SZ
正在处理002768.SZ
正在处理002769.SZ
正在处理002770.SZ
正在处理002771.SZ
正在处理002772.SZ
正在处理002773.SZ
正在处理002774.SZ
正在处理002775.SZ
正在处理002776.SZ
正在处理002777.SZ
正在处理002778.SZ
正在处理002779.SZ
正在处理002780.SZ
正在处理002781.SZ
正在处理002782.SZ
正在处理002783.SZ
正在处理002785.SZ
正在处理002786.SZ
正在处理002787.SZ
正在处理002788.SZ
正在处理002789.SZ
正在处理002790.SZ
正在处理002791.SZ
正在处理002792.SZ
正在处理002793.SZ
正在处理002795.SZ
正在处理002796.SZ
正在处理002797.SZ
正在处理002798.SZ
正在处理002799.SZ
正在处理002800.SZ
正在处理00

正在处理300360.SZ
正在处理300362.SZ
正在处理300363.SZ
正在处理300364.SZ
正在处理300365.SZ
正在处理300366.SZ
正在处理300367.SZ
正在处理300368.SZ
正在处理300369.SZ
正在处理300370.SZ
正在处理300371.SZ
正在处理300373.SZ
正在处理300374.SZ
正在处理300375.SZ
正在处理300376.SZ
正在处理300377.SZ
正在处理300378.SZ
正在处理300379.SZ
正在处理300380.SZ
正在处理300381.SZ
正在处理300382.SZ
正在处理300383.SZ
正在处理300384.SZ
正在处理300385.SZ
正在处理300386.SZ
正在处理300387.SZ
正在处理300388.SZ
正在处理300389.SZ
正在处理300390.SZ
正在处理300391.SZ
正在处理300392.SZ
正在处理300393.SZ
正在处理300394.SZ
正在处理300395.SZ
正在处理300396.SZ
正在处理300397.SZ
正在处理300398.SZ
正在处理300399.SZ
正在处理300400.SZ
正在处理300401.SZ
正在处理300402.SZ
正在处理300403.SZ
正在处理300404.SZ
正在处理300405.SZ
正在处理300406.SZ
正在处理300407.SZ
正在处理300408.SZ
正在处理300409.SZ
正在处理300410.SZ
正在处理300411.SZ
正在处理300412.SZ
正在处理300413.SZ
正在处理300414.SZ
正在处理300415.SZ
正在处理300416.SZ
正在处理300417.SZ
正在处理300418.SZ
正在处理300419.SZ
正在处理300420.SZ
正在处理300421.SZ
正在处理300422.SZ
正在处理300423.SZ
正在处理300424.SZ
正在处理300425.SZ
正在处理300426.SZ
正在处理300427.SZ
正在处理300428.SZ
正在处理300429.SZ
正在处理300430.SZ
正在处理300431.SZ
正在处理300432.SZ
正在处理30

正在处理600179.SH
正在处理600180.SH
正在处理600182.SH
正在处理600183.SH
正在处理600184.SH
正在处理600185.SH
正在处理600186.SH
正在处理600187.SH
正在处理600188.SH
正在处理600189.SH
正在处理600190.SH
正在处理600191.SH
正在处理600192.SH
正在处理600193.SH
正在处理600195.SH
正在处理600196.SH
正在处理600197.SH
正在处理600198.SH
正在处理600199.SH
正在处理600200.SH
正在处理600201.SH
正在处理600202.SH
正在处理600203.SH
正在处理600206.SH
正在处理600207.SH
正在处理600208.SH
正在处理600209.SH
正在处理600210.SH
正在处理600211.SH
正在处理600212.SH
正在处理600213.SH
正在处理600215.SH
正在处理600216.SH
正在处理600217.SH
正在处理600218.SH
正在处理600219.SH
正在处理600220.SH
正在处理600221.SH
正在处理600222.SH
正在处理600223.SH
正在处理600225.SH
正在处理600226.SH
正在处理600227.SH
正在处理600228.SH
正在处理600229.SH
正在处理600230.SH
正在处理600231.SH
正在处理600232.SH
正在处理600233.SH
正在处理600234.SH
正在处理600235.SH
正在处理600236.SH
正在处理600237.SH
正在处理600238.SH
正在处理600239.SH
正在处理600241.SH
正在处理600242.SH
正在处理600243.SH
正在处理600246.SH
正在处理600247.SH
正在处理600248.SH
正在处理600249.SH
正在处理600250.SH
正在处理600251.SH
正在处理600252.SH
正在处理600255.SH
正在处理600256.SH
正在处理600257.SH
正在处理600258.SH
正在处理600259.SH
正在处理600260.SH
正在处理60

正在处理600874.SH
正在处理600875.SH
正在处理600876.SH
正在处理600877.SH
正在处理600879.SH
正在处理600880.SH
正在处理600881.SH
正在处理600882.SH
正在处理600883.SH
正在处理600884.SH
正在处理600885.SH
正在处理600886.SH
正在处理600887.SH
正在处理600888.SH
正在处理600889.SH
正在处理600890.SH
正在处理600891.SH
正在处理600892.SH
正在处理600893.SH
正在处理600894.SH
正在处理600895.SH
正在处理600896.SH
正在处理600897.SH
正在处理600898.SH
正在处理600900.SH
正在处理600901.SH
正在处理600903.SH
正在处理600908.SH
正在处理600909.SH
正在处理600917.SH
正在处理600919.SH
正在处理600926.SH
正在处理600928.SH
正在处理600929.SH
正在处理600933.SH
正在处理600936.SH
正在处理600939.SH
正在处理600958.SH
正在处理600959.SH
正在处理600960.SH
正在处理600961.SH
正在处理600962.SH
正在处理600963.SH
正在处理600965.SH
正在处理600966.SH
正在处理600967.SH
正在处理600968.SH
正在处理600969.SH
正在处理600970.SH
正在处理600971.SH
正在处理600973.SH
正在处理600975.SH
正在处理600976.SH
正在处理600977.SH
正在处理600978.SH
正在处理600979.SH
正在处理600980.SH
正在处理600981.SH
正在处理600982.SH
正在处理600983.SH
正在处理600984.SH
正在处理600985.SH
正在处理600986.SH
正在处理600987.SH
正在处理600988.SH
正在处理600989.SH
正在处理600990.SH
正在处理600992.SH
正在处理600993.SH
正在处理600995.SH
正在处理600996.SH
正在处理60

正在处理603666.SH
正在处理603667.SH
正在处理603668.SH
正在处理603669.SH
正在处理603676.SH
正在处理603677.SH
正在处理603678.SH
正在处理603679.SH
正在处理603680.SH
正在处理603681.SH
正在处理603683.SH
正在处理603685.SH
正在处理603686.SH
正在处理603687.SH
正在处理603688.SH
正在处理603689.SH
正在处理603690.SH
正在处理603693.SH
正在处理603696.SH
正在处理603697.SH
正在处理603698.SH
正在处理603699.SH
正在处理603700.SH
正在处理603701.SH
正在处理603703.SH
正在处理603706.SH
正在处理603707.SH
正在处理603708.SH
正在处理603709.SH
正在处理603711.SH
正在处理603712.SH
正在处理603713.SH
正在处理603716.SH
正在处理603717.SH
正在处理603718.SH
正在处理603721.SH
正在处理603722.SH
正在处理603725.SH
正在处理603726.SH
正在处理603727.SH
正在处理603728.SH
正在处理603729.SH
正在处理603730.SH
正在处理603733.SH
正在处理603737.SH
正在处理603738.SH
正在处理603739.SH
正在处理603755.SH
正在处理603757.SH
正在处理603758.SH
正在处理603766.SH
正在处理603767.SH
正在处理603768.SH
正在处理603773.SH
正在处理603776.SH
正在处理603777.SH
正在处理603778.SH
正在处理603779.SH
正在处理603786.SH
正在处理603787.SH
正在处理603788.SH
正在处理603789.SH
正在处理603790.SH
正在处理603797.SH
正在处理603798.SH
正在处理603799.SH
正在处理603800.SH
正在处理603801.SH
正在处理603803.SH
正在处理603806.SH
正在处理603808.SH
正在处理60

#####  查看并存储数据

In [12]:
date_result.info()
date_result.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28269 entries, 0 to 28268
Data columns (total 14 columns):
trade_date    28269 non-null object
ts_code       28269 non-null object
name          28269 non-null object
close         28269 non-null float64
pct_chg       28269 non-null float64
amp           28269 non-null float64
fc_ratio      28269 non-null float64
fl_ratio      28269 non-null float64
fd_amount     28269 non-null float64
first_time    28269 non-null object
last_time     28269 non-null object
open_times    28269 non-null int64
strth         28269 non-null float64
limit         28269 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 3.0+ MB


,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20190102,600399.SH,*ST抚钢,2.63,5.20,0.0,2024.7509,0.3159,16382665.0,09:25:04,09:25:04,0,100.00000,U
1,20190102,600610.SH,*ST毅达,1.81,5.23,0.0,543.7526,1.0417,7081263.0,09:25:04,09:25:04,0,84.03134,U
2,20190102,300756.SZ,中山金马,85.32,10.01,0.0,291.2599,4.2432,36203152.0,09:25:03,09:25:03,0,75.19410,U


In [13]:
stock_result.info()
stock_result.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27720 entries, 0 to 27719
Data columns (total 14 columns):
trade_date    27720 non-null object
ts_code       27720 non-null object
name          27720 non-null object
close         27720 non-null float64
pct_chg       27720 non-null float64
amp           27720 non-null float64
fc_ratio      27720 non-null float64
fl_ratio      27720 non-null float64
fd_amount     27720 non-null float64
first_time    27720 non-null object
last_time     27720 non-null object
open_times    27720 non-null int64
strth         27720 non-null float64
limit         27720 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 3.0+ MB


,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20200203,000001.SZ,平安银行,13.99,-9.97,4.57,0.3998,0.0047,12800850.0,09:25:03,14:52:33,43,55.470220,D
1,20200203,000004.SZ,国农科技,20.54,-9.99,0.00,408.8721,2.1285,36112404.0,09:25:03,09:25:03,0,79.310524,D
2,20191107,000004.SZ,国农科技,20.88,10.01,0.00,95.4751,1.9584,33775864.0,09:25:03,09:25:03,0,68.341630,U


In [14]:
datefilepath=out_csv_dir/'date_result.csv'
date_result.to_csv(str(datefilepath),index=False) 
stockfilepath=out_csv_dir/'stock_result.csv'
stock_result.to_csv(str(stockfilepath),index=False) 

##### 未涨跌停过的股票

In [15]:
nolimit=set(Slist.ts_code)-set(limit_stcok)  #虽然不包含这期间退市的股票，但是它们都连续跌停，因此并不影响结果
print('在{}至{}期间，共有{}只股票没涨跌停'.format(sdate,edate,len(nolimit))) 
list(nolimit)[:20]

在20190101至20200211期间，共有134只股票没涨跌停


['600176.SH',
 '601988.SH',
 '600177.SH',
 '000786.SZ',
 '603939.SH',
 '002242.SZ',
 '300144.SZ',
 '603139.SH',
 '600145.SH',
 '600285.SH',
 '600597.SH',
 '600062.SH',
 '600741.SH',
 '601288.SH',
 '600761.SH',
 '600674.SH',
 '600196.SH',
 '600309.SH',
 '601877.SH',
 '601158.SH']

#### 开板情况

##### 涨停开板

In [55]:
# df0205=date_result[(date_result.trade_date == '20200205')
#            & (date_result.open_times>10) &(date_result.open_times<19)& (date_result.limit == 'U')]
# df0205.sort_values(by='open_times',ascending=False)
df0205 = date_result[(date_result.trade_date == '20200205')
                     & (date_result.limit == 'U')]
limit0205=df0205.drop(columns=['close','pct_chg','limit','trade_date'])

In [31]:
df0207 = pro.limit_list(trade_date='20200207')
df07 = df0207[(df0207.open_times > 4) & (df0207.limit == 'U')]
df07.sort_values(by='open_times', ascending=False)

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
50,20200207,300160.SZ,秀强股份,5.81,10.04,5.87,1.9780,0.3075,10381889.0,09:25:03,14:44:21,126,59.343174,U
182,20200207,300026.SZ,红日药业,6.22,10.09,9.20,0.6188,0.0856,12956496.0,09:32:06,14:55:03,113,15.866159,U
123,20200207,300152.SZ,科融环境,3.34,9.87,7.89,2.9393,0.2076,4942558.5,10:09:03,13:46:42,98,52.414314,U
37,20200207,300459.SZ,金科文化,4.38,10.05,4.27,2.9357,0.3267,32467456.0,09:32:06,13:23:54,83,61.039910,U
72,20200207,002693.SZ,双成药业,6.75,9.93,8.47,1.9428,0.2419,6586522.0,09:33:00,14:52:24,83,56.736370,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,20200207,603083.SH,剑桥科技,31.48,9.99,9.50,4.1711,0.7063,24938864.0,10:30:22,13:42:26,6,55.952427,U
140,20200207,603416.SH,信捷电气,30.20,10.02,10.93,2.1142,0.0760,2093464.0,10:22:54,14:53:26,5,49.285217,U
142,20200207,600751.SH,海航科技,2.87,9.96,11.49,6.8765,0.1711,12634486.0,10:23:56,11:00:59,5,49.206207,U
181,20200207,002308.SZ,威创股份,5.69,10.06,9.09,0.1605,0.0164,843827.0,10:08:45,14:56:51,5,15.926873,U


In [32]:
df07[df07.ts_code=='603157.SH']    

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
168,20200207,603157.SH,拉夏贝尔,5.1,9.91,13.58,5.6227,0.7626,5670685.0,14:10:54,14:46:27,21,47.0204,U


#### 日线行情

In [44]:
daily_data=[]
for date in trade_cal:
    while(True):
        try :
            df_daily=pro.daily(trade_date=date)
            daily_data.append(df_daily)
            print("已下载{}的日线数据".format(date))
            break
        except:
            print('超时，等待10秒')
            time.sleep(10)
daily=pd.concat(daily_data,ignore_index=True)            
print('OK')

已下载20190102的日线数据
已下载20190103的日线数据
已下载20190104的日线数据
已下载20190107的日线数据
已下载20190108的日线数据
已下载20190109的日线数据
已下载20190110的日线数据
已下载20190111的日线数据
已下载20190114的日线数据
已下载20190115的日线数据
已下载20190116的日线数据
已下载20190117的日线数据
已下载20190118的日线数据
已下载20190121的日线数据
已下载20190122的日线数据
已下载20190123的日线数据
已下载20190124的日线数据
已下载20190125的日线数据
已下载20190128的日线数据
已下载20190129的日线数据
已下载20190130的日线数据
已下载20190131的日线数据
已下载20190201的日线数据
已下载20190211的日线数据
已下载20190212的日线数据
已下载20190213的日线数据
已下载20190214的日线数据
已下载20190215的日线数据
已下载20190218的日线数据
已下载20190219的日线数据
已下载20190220的日线数据
已下载20190221的日线数据
已下载20190222的日线数据
已下载20190225的日线数据
已下载20190226的日线数据
已下载20190227的日线数据
已下载20190228的日线数据
已下载20190301的日线数据
已下载20190304的日线数据
已下载20190305的日线数据
已下载20190306的日线数据
已下载20190307的日线数据
已下载20190308的日线数据
已下载20190311的日线数据
已下载20190312的日线数据
已下载20190313的日线数据
已下载20190314的日线数据
已下载20190315的日线数据
已下载20190318的日线数据
已下载20190319的日线数据
已下载20190320的日线数据
已下载20190321的日线数据
已下载20190322的日线数据
已下载20190325的日线数据
已下载20190326的日线数据
已下载20190327的日线数据
已下载20190328的日线数据
已下载20190329的日线数据
已下载20190401的日线

In [45]:
daily.info()
daily.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963613 entries, 0 to 963612
Data columns (total 11 columns):
ts_code       963613 non-null object
trade_date    963613 non-null object
open          963613 non-null float64
high          963613 non-null float64
low           963613 non-null float64
close         963613 non-null float64
pre_close     963613 non-null float64
change        963613 non-null float64
pct_chg       963613 non-null float64
vol           963613 non-null float64
amount        963613 non-null float64
dtypes: float64(9), object(2)
memory usage: 80.9+ MB


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000671.SZ,20190102,5.20,5.24,5.09,5.15,5.19,-0.04,-0.7707,105582.09,54330.033
1,000672.SZ,20190102,8.45,8.52,8.37,8.42,8.44,-0.02,-0.2370,36908.97,31156.758
2,000673.SZ,20190102,4.73,4.79,4.66,4.69,4.69,0.00,0.0000,94097.77,44322.224


In [59]:
daily020607=daily[(daily.trade_date=='20200206')|(daily.trade_date=='20200206')]
daily020607.head(3)
daily020607.tail(3)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
956082,002115.SZ,20200206,10.90,11.75,10.90,11.43,10.95,0.48,4.3836,746723.55,844185.151
956083,002120.SZ,20200206,32.13,34.56,31.40,33.78,32.13,1.65,5.1354,173732.41,580295.169
956084,002117.SZ,20200206,10.45,11.46,10.32,11.40,10.42,0.98,9.4050,286064.16,317857.495


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
959844,300583.SZ,20200206,33.00,35.59,32.50,35.59,32.35,3.24,10.0155,49354.08,170818.445
959845,603826.SH,20200206,20.58,21.14,20.12,20.71,20.58,0.13,0.6317,42527.74,88306.200
959846,002868.SZ,20200206,15.60,16.13,15.58,16.06,15.57,0.49,3.1471,12231.82,19524.824


In [61]:
dlimit=pd.merge(limit0205,daily020607,on='ts_code')
dlimit[dlimit.pct_chg<0]

,ts_code,name,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
10,688298.SH,N东方,301.13,0.0124,0.0000,233568.0,09:25:04,09:25:04,0,100.000000,20200206,120.00,147.77,117.10,128.00,145.98,-17.98,-12.3168,145404.73,1923537.133
31,002836.SZ,新宏泽,9.56,129.1502,0.7436,12182528.0,09:37:36,09:37:42,1,63.392197,20200206,10.68,10.68,10.17,10.22,10.24,-0.02,-0.1953,22947.05,23699.805
41,300499.SZ,高澜股份,4.77,3.4685,0.4401,10379622.0,09:25:03,14:47:12,32,60.598858,20200206,18.22,20.60,18.01,18.99,19.14,-0.15,-0.7837,259442.20,493178.606
122,603825.SH,华扬联众,13.94,2.3493,0.4326,12068390.0,10:23:55,14:29:48,3,51.316566,20200206,29.50,31.55,27.71,28.33,29.43,-1.10,-3.7377,194720.12,580201.159
130,002323.SZ,ST百特,9.72,3.8957,0.1949,1603430.1,09:25:00,13:29:33,76,50.679317,20200206,2.51,2.67,2.46,2.53,2.59,-0.06,-2.3166,191145.45,48002.748
136,600436.SH,片仔癀,9.72,2.0142,0.0317,25932342.0,14:06:10,14:39:49,15,50.483000,20200206,137.50,137.73,131.97,134.29,135.77,-1.48,-1.0901,102340.61,1372821.576
148,000623.SZ,吉林敖东,10.50,2.3058,0.0716,13962662.0,14:45:12,14:49:33,7,49.734160,20200206,16.98,17.47,16.60,16.74,17.07,-0.33,-1.9332,673113.89,1152995.775
171,600654.SH,*ST中安,5.76,0.8431,0.0150,227532.0,10:32:36,14:55:22,25,14.325057,20200206,2.01,2.04,1.95,1.99,2.01,-0.02,-0.9950,103878.02,20672.226


In [37]:
#pro=ts.pro_api()

lastday='20200131'
firstday='20190101'
all_day=pro.trade_cal().query('is_open==1 and cal_date<=@lastday and \
                                 cal_date>=@firstday')['cal_date']

dflst=[]
for day in all_day:
    while(True):
        try:
            dflst.append(pro.limit_list(trade_date=day))
            print(day)
            break
        except:
            print('因超时等待10秒重试......')
            t.sleep(10)
dfl=pd.concat(dflst,sort=False)
print('All done.')

20190102
20190103
20190104
20190107
20190108
20190109
20190110
20190111
20190114
20190115
20190116
20190117
20190118
20190121
20190122
20190123
20190124
20190125
20190128
20190129
20190130
20190131
20190201
20190211
20190212
20190213
20190214
20190215
20190218
20190219
20190220
20190221
20190222
20190225
20190226
20190227
20190228
20190301
20190304
20190305
20190306
20190307
20190308
20190311
20190312
20190313
20190314
20190315
20190318
20190319
20190320
20190321
20190322
20190325
20190326
20190327
20190328
20190329
20190401
20190402
20190403
20190404
20190408
20190409
20190410
20190411
20190412
20190415
20190416
20190417
20190418
20190419
20190422
20190423
20190424
20190425
20190426
20190429
20190430
20190506
20190507
20190508
20190509
20190510
20190513
20190514
20190515
20190516
20190517
20190520
20190521
20190522
20190523
20190524
20190527
20190528
20190529
20190530
20190531
20190603
20190604
20190605
20190606
20190610
20190611
20190612
20190613
20190614
20190617
20190618
20190619
2

In [42]:
set(result.ts_code)-set(Slist.ts_code)

{'000018.SZ',
 '000043.SZ',
 '000693.SZ',
 '000939.SZ',
 '000995.SZ',
 '002018.SZ',
 '002070.SZ',
 '002143.SZ',
 '002260.SZ',
 '002477.SZ',
 '002604.SZ',
 '002680.SZ',
 '300104.SZ',
 '300216.SZ',
 '600074.SH',
 '600401.SH',
 '600610.SH',
 '600747.SH'}

In [22]:
mytest

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20191107,000004.SZ,国农科技,20.88,10.01,0.00,95.4751,1.9584,33775864.0,09:25:03,09:25:03,0,68.341630,U
1,20190919,000004.SZ,国农科技,22.00,10.00,8.90,21.4926,1.4702,26716448.0,,,0,0.000000,U
2,20190506,000004.SZ,国农科技,22.75,-10.01,8.50,0.0000,0.0000,0.0,09:33:27,14:52:03,13,51.500000,D
3,20190430,000004.SZ,国农科技,25.28,10.01,10.97,4.1339,0.1489,3121650.2,10:05:51,14:52:51,31,49.450832,U
4,20190410,000004.SZ,国农科技,23.84,10.01,0.00,0.2913,0.0012,23840.0,09:25:03,09:25:03,0,65.010190,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20191014,688368.SH,N晶丰,104.36,84.12,23.09,0.1583,0.0002,1606517.9,09:25:00,09:25:00,0,100.000000,U
0,20191031,688369.SH,N致远,76.90,55.70,24.76,0.1120,0.0002,1174032.4,09:25:03,09:25:03,0,100.000000,U
0,20190722,688388.SH,嘉元科技,56.66,100.50,140.94,0.3722,0.0006,7846503.5,,,0,0.000000,U
0,20191105,688389.SH,N普门,19.63,115.71,42.64,0.2979,0.0002,1833540.1,09:25:03,09:25:03,0,100.000000,U


In [30]:
test=pd.merge(result,mytest,on=['trade_date','ts_code'],how='left')
test[test.name_y.isnull()]

,trade_date,ts_code,name_x,close_x,pct_chg_x,amp_x,fc_ratio_x,fl_ratio_x,fd_amount_x,first_time_x,last_time_x,open_times_x,strth_x,limit_x,name_y,close_y,pct_chg_y,amp_y,fc_ratio_y,fl_ratio_y,fd_amount_y,first_time_y,last_time_y,open_times_y,strth_y,limit_y
1,20190102,600610.SH,*ST毅达,1.81,5.23,0.00,543.7526,1.0417,7081263.0,09:25:04,09:25:04,0,84.031340,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,20190102,300104.SZ,乐视网,2.74,10.04,2.01,25.1338,0.6429,54138388.0,09:25:03,09:30:21,2,60.535526,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,20190102,000995.SZ,*ST皇台,4.98,-4.96,4.58,1.8309,0.0438,386946.0,09:25:03,14:34:30,20,60.605278,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,20190103,600610.SH,*ST毅达,1.90,4.97,0.00,932.8632,1.2154,8672550.0,09:25:04,09:25:04,0,97.650215,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,20190103,300104.SZ,乐视网,3.01,9.85,9.85,3.5980,0.4248,39292364.0,14:46:54,14:47:00,1,55.531036,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20921,20191203,000018.SZ,神城A退,0.40,-9.09,0.00,459456.8000,8.3044,33080890.0,09:49:39,09:49:39,0,100.000000,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20968,20191204,000018.SZ,神城A退,0.36,-10.00,0.00,227272.8300,7.6679,27490920.0,09:33:12,09:33:12,0,100.000000,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21019,20191205,000018.SZ,神城A退,0.32,-11.11,0.00,6949.1150,7.3532,23433528.0,09:25:03,09:25:03,0,100.000000,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21081,20191206,000018.SZ,神城A退,0.29,-9.38,0.00,5765.4427,7.1734,20717484.0,09:25:03,09:25:03,0,100.000000,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
mytest[mytest.trade_date=='20190102']

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
8,20190102,000056.SZ,皇庭国际,5.20,9.94,10.15,6.7042,0.1381,6.488336e+06,14:23:39,14:37:54,6,50.527325,U
21,20190102,000070.SZ,特发信息,7.81,10.00,11.13,3.0394,0.1858,8.708931e+06,10:28:45,14:05:39,62,49.183228,U
16,20190102,000533.SZ,万 家 乐,2.78,9.88,8.70,17.4469,0.7017,1.336113e+07,09:44:09,09:44:09,0,58.079780,U
2,20190102,000540.SZ,中天金融,4.38,-10.06,0.00,3476.3730,5.0594,1.441646e+09,09:25:03,09:25:03,0,100.000000,D
8,20190102,000592.SZ,平潭发展,2.94,10.11,7.87,3.6469,0.1546,8.697108e+06,10:06:51,13:58:24,20,52.502415,U
1,20190102,000755.SZ,山西路桥,4.38,10.05,10.05,56.5346,0.8242,1.693965e+07,10:43:48,10:43:48,0,60.644665,U
1,20190102,000963.SZ,华东医药,23.81,-10.02,10.92,0.1977,0.0041,1.283359e+06,13:48:51,15:00:03,17,9.099976,D
29,20190102,002017.SZ,东信和平,10.24,9.99,14.18,3.3914,0.3917,1.384386e+07,09:55:51,14:07:09,9,46.178730,U
36,20190102,002021.SZ,中捷资源,2.72,-9.93,7.62,3.9790,0.1336,2.499027e+06,10:13:03,14:40:12,16,52.784576,D
3,20190102,002075.SZ,沙钢股份,7.23,-9.96,9.46,18.7212,0.5132,8.188452e+07,13:00:54,13:28:30,19,52.437782,D


In [43]:
dfl

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20190102,600399.SH,*ST抚钢,2.63,5.20,0.00,2024.7509,0.3159,16382665.0,09:25:04,09:25:04,0,100.000000,U
1,20190102,600610.SH,*ST毅达,1.81,5.23,0.00,543.7526,1.0417,7081263.0,09:25:04,09:25:04,0,84.031340,U
2,20190102,300756.SZ,中山金马,85.32,10.01,0.00,291.2599,4.2432,36203152.0,09:25:03,09:25:03,0,75.194100,U
3,20190102,600605.SH,汇通能源,10.70,9.97,0.00,210.9398,1.8058,28468366.0,09:25:06,09:25:06,0,72.382900,U
4,20190102,002496.SZ,ST辉丰,2.02,5.21,3.65,72.4797,0.3282,6279331.5,09:30:54,09:31:00,1,68.614380,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,20200123,300192.SZ,科斯伍德,13.79,-9.99,8.55,0.7848,0.0660,1567923.0,14:38:24,14:55:54,2,11.531780,D
141,20200123,600666.SH,*ST瑞德,2.28,-5.00,8.75,0.6624,0.0241,442274.4,10:40:33,14:55:56,45,11.317451,D
142,20200123,300078.SZ,思创医惠,12.03,-10.02,10.77,0.1472,0.0063,555786.0,14:18:36,15:00:03,22,9.245037,D
143,20200123,002323.SZ,ST百特,2.74,-4.86,0.00,621.6091,2.2307,19413174.0,,,0,0.000000,D


In [32]:
#dfcal = pro.trade_cal(is_open=1, start_date=sdate, end_date=edate)['cal_date']
#dfcal
all_day=pro.trade_cal().query('is_open==1 and cal_date<=@edate and \
                                 cal_date>=@sdate')['cal_date']
all_day

10241    20190102
10242    20190103
10243    20190104
10246    20190107
10247    20190108
           ...   
10617    20200113
10618    20200114
10619    20200115
10620    20200116
10621    20200117
Name: cal_date, Length: 256, dtype: object

#### 涨跌停数据基本情况

In [22]:
limit_data.info()
v=[]  
v=np.random.randint(len(limit_data.index),size=10)
limit_data.iloc[v]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22772 entries, 0 to 22771
Data columns (total 14 columns):
trade_date    22772 non-null object
ts_code       22772 non-null object
name          22772 non-null object
close         22772 non-null float64
pct_chg       22772 non-null float64
amp           22772 non-null float64
fc_ratio      22772 non-null float64
fl_ratio      22772 non-null float64
fd_amount     22772 non-null float64
first_time    22772 non-null object
last_time     22772 non-null object
open_times    22772 non-null int64
strth         22772 non-null float64
limit         22772 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 2.4+ MB


,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
9964,20190305,300128.SZ,锦富技术,5.60,10.02,12.38,5.5430,0.6248,3.728605e+07,10:06:54,10:06:54,0,53.205540,U
7938,20190513,002766.SZ,*ST索菱,3.71,-4.87,0.00,10053.4270,8.6068,7.944520e+07,09:25:03,09:25:03,0,100.000000,D
8462,20190225,002876.SZ,三利谱,47.74,10.00,9.77,5.0934,0.2079,5.428038e+06,14:21:45,14:54:09,6,50.749733,U
14728,20200115,300811.SZ,铂科新材,107.76,10.00,0.00,19.4512,5.7877,8.980998e+07,,,0,0.000000,U
4892,20190318,002302.SZ,西部建设,13.61,10.02,10.02,5.9398,0.3036,5.119777e+07,14:05:33,14:05:33,0,55.589160,U
9952,20190606,300127.SZ,银河磁体,18.02,10.01,12.27,37.3094,2.5457,1.041845e+08,09:40:30,09:40:30,0,56.588223,U
1840,20190510,000811.SZ,冰轮环境,8.69,10.00,9.37,0.3420,0.0130,7.377809e+05,14:49:36,14:55:15,3,10.664848,U
11435,20190107,300345.SZ,红宇新材,4.10,9.92,6.70,54.3556,1.1560,1.747188e+07,09:30:24,09:30:24,0,63.793358,U
3719,20190425,002140.SZ,东华科技,12.06,10.04,9.67,1.5512,0.1709,9.060678e+06,09:34:33,14:49:51,59,55.493668,U
3113,20190311,002058.SZ,威 尔 泰,14.01,9.97,9.11,7.6118,0.3045,6.119288e+06,13:49:09,13:51:54,4,51.666405,U


在20190101至20200118期间，共有627只股票没涨跌停


['600811.SH',
 '002457.SZ',
 '601966.SH',
 '002203.SZ',
 '601991.SH',
 '002173.SZ',
 '603308.SH',
 '603728.SH',
 '603955.SH',
 '002344.SZ',
 '600011.SH',
 '600919.SH',
 '002749.SZ',
 '603897.SH',
 '601333.SH',
 '002038.SZ',
 '002061.SZ',
 '688089.SH',
 '601636.SH',
 '600723.SH']

In [13]:
csvfilepath=out_csv_dir/'limit_data.csv'
limit_data=pd.read_csv(str(csvfilepath))
limit_data

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20191107,000004.SZ,国农科技,20.88,10.01,0.00,95.4751,1.9584,33775864.0,09:25:03,09:25:03,0,68.341630,U
1,20190919,000004.SZ,国农科技,22.00,10.00,8.90,21.4926,1.4702,26716448.0,NaN,NaN,0,0.000000,U
2,20190506,000004.SZ,国农科技,22.75,-10.01,8.50,0.0000,0.0000,0.0,09:33:27,14:52:03,13,51.500000,D
3,20190430,000004.SZ,国农科技,25.28,10.01,10.97,4.1339,0.1489,3121650.2,10:05:51,14:52:51,31,49.450832,U
4,20190410,000004.SZ,国农科技,23.84,10.01,0.00,0.2913,0.0012,23840.0,09:25:03,09:25:03,0,65.010190,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22767,20191014,688368.SH,N晶丰,104.36,84.12,23.09,0.1583,0.0002,1606517.9,09:25:00,09:25:00,0,100.000000,U
22768,20191031,688369.SH,N致远,76.90,55.70,24.76,0.1120,0.0002,1174032.4,09:25:03,09:25:03,0,100.000000,U
22769,20190722,688388.SH,嘉元科技,56.66,100.50,140.94,0.3722,0.0006,7846503.5,NaN,NaN,0,0.000000,U
22770,20191105,688389.SH,N普门,19.63,115.71,42.64,0.2979,0.0002,1833540.1,09:25:03,09:25:03,0,100.000000,U


In [ ]:
limit_data[]

In [114]:

pro.limit_list(ts_code='600388.SH',start_date=sdate, end_date=edate)

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit


In [15]:
limit_data[(limit_data.open_times>10)&(limit_data.limit=='U')]

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
3,20190430,000004.SZ,国农科技,25.28,10.01,10.97,4.1339,0.1489,3121650.2,10:05:51,14:52:51,31,49.450832,U
5,20190325,000004.SZ,国农科技,21.67,10.00,5.89,2.4241,0.1332,2396702.0,09:33:09,14:53:18,23,54.359070,U
7,20190903,000005.SZ,世纪星源,3.48,10.13,10.13,3.8327,0.3445,12683556.0,10:37:21,14:47:12,25,50.270493,U
22,20190225,000008.SZ,神州高铁,4.42,9.95,9.95,0.6532,0.0249,2811937.8,13:50:12,14:05:00,16,50.116560,U
26,20190819,000009.SZ,中国宝安,4.74,9.98,5.57,5.6314,0.1554,18738452.0,09:32:00,13:13:27,27,55.000908,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22656,20191219,603998.SH,方盛制药,8.56,10.03,9.38,3.8398,0.2608,9493896.0,13:53:51,14:18:45,15,51.017030,U
22657,20191008,603998.SH,方盛制药,7.90,10.03,9.33,11.4474,0.3942,13241980.0,09:45:07,10:03:34,21,51.834454,U
22658,20190910,603998.SH,方盛制药,8.47,10.00,11.56,5.7050,0.5075,18275720.0,11:22:25,13:04:35,11,49.035870,U
22659,20190826,603998.SH,方盛制药,7.95,9.96,13.14,1.1053,0.1299,4389990.0,10:40:57,14:34:25,82,51.977028,U


#### 涨跌停强度分析
+ 接口：limit_list
+ 描述：获取每日涨跌停股票统计，包括封闭时间和打开次数等数据，帮助用户快速定位近期强（弱）势股，以及研究超短线策略。
+ 限量：单次最大1000，总量不限制

In [20]:

#获取时间段统计信息-2019年至今
dfll = pro.limit_list(start_date=sdate, end_date=edate,limit_type='U')
dfll[(dfll.trade_date=='20200327')&(dfll.ts_code=='603157.SH')]
#dfll.columns=llCname
#dfll[dfll.open_times>20].sort_values(by='open_times',ascending=False)

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
26,20200327,603157.SH,拉夏贝尔,4.49,10.05,8.33,45.6151,2.2248,14564513.0,09:41:25,09:53:34,1,56.342747,U


ValueError: Length mismatch: Expected axis has 14 elements, new values have 10 elements

In [33]:
df1227=pro.limit_list(trade_date='20191227',limit_type='U')
df1227

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20191227,002972.SZ,N科安达,16.55,44.04,24.02,4574.9153,15.2450,111215744.0,09:25:03,09:25:03,0,100.000000,U
1,20191227,300807.SZ,天迈科技,45.11,10.00,0.00,5250.9064,32.7752,251343408.0,09:25:03,09:25:03,0,100.000000,U
2,20191227,300173.SZ,智慧松德,6.49,10.00,0.00,625.5034,4.1249,117704736.0,09:25:03,09:25:03,0,86.892620,U
3,20191227,603598.SH,引力传媒,14.26,10.03,0.00,600.8521,4.5545,173967728.0,09:25:04,09:25:04,0,86.029820,U
4,20191227,002163.SZ,中航三鑫,5.16,10.02,0.00,440.5163,1.5673,64977816.0,09:25:03,09:25:03,0,80.418070,U
5,20191227,000927.SZ,一汽夏利,5.83,10.00,0.00,258.5021,1.4198,132043528.0,09:25:03,09:25:03,0,74.047570,U
6,20191227,000955.SZ,欣龙控股,6.77,10.08,0.00,230.6344,4.1297,150303344.0,09:25:03,09:25:03,0,73.072205,U
7,20191227,600485.SH,*ST信威,2.77,4.92,0.00,200.4771,0.8790,46139556.0,09:25:02,09:25:02,0,72.016700,U
8,20191227,300787.SZ,海能实业,90.38,10.00,6.91,47.0713,5.5294,106047280.0,10:19:00,10:19:00,0,67.897125,U
9,20191227,600652.SH,*ST游久,2.27,5.09,1.39,40.6434,0.3816,7212986.0,09:25:04,09:25:04,0,67.693410,U


#### 查看涨跌停价格


In [69]:
#获取单个股票数据
df07limit = pro.stk_limit(ts_code='603157.SH', start_date='20190724', end_date='20200117')
df07limit

,trade_date,ts_code,up_limit,down_limit
0,20200117,603157.SH,7.28,5.96
1,20200116,603157.SH,6.62,5.42
2,20200115,603157.SH,6.55,5.36
3,20200114,603157.SH,6.51,5.33
4,20200113,603157.SH,6.49,5.31
...,...,...,...,...
116,20190730,603157.SH,6.70,5.48
117,20190729,603157.SH,6.71,5.49
118,20190726,603157.SH,6.68,5.46
119,20190725,603157.SH,6.71,5.49


In [7]:
df07daily = pro.daily(ts_code='603157.SH', start_date='20190724', end_date='20200117')
df07daily

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,603157.SH,20200117,6.44,6.44,5.96,6.08,6.62,-0.54,-8.1571,283475.93,174131.725
1,603157.SH,20200116,6.10,6.62,6.08,6.62,6.02,0.60,9.9668,376776.85,246544.025
2,603157.SH,20200115,5.96,6.19,5.87,6.02,5.95,0.07,1.1765,136810.00,83169.196
3,603157.SH,20200114,6.05,6.10,5.76,5.95,5.92,0.03,0.5068,146250.43,86685.683
4,603157.SH,20200113,6.00,6.09,5.86,5.92,5.90,0.02,0.3390,103887.92,61970.398
...,...,...,...,...,...,...,...,...,...,...,...
116,603157.SH,20190730,6.09,6.16,6.06,6.09,6.09,0.00,0.0000,20061.21,12270.988
117,603157.SH,20190729,6.10,6.13,6.05,6.09,6.10,-0.01,-0.1639,17948.67,10911.368
118,603157.SH,20190726,6.07,6.13,6.02,6.10,6.07,0.03,0.4942,21577.00,13131.268
119,603157.SH,20190725,6.11,6.13,6.04,6.07,6.10,-0.03,-0.4918,27393.46,16656.479


In [71]:
df07daily.drop(columns=['change','pct_chg','vol','amount'],inplace=True)
df07=pd.merge(df07daily,df07limit,on='trade_date')
df07.drop(columns=['ts_code_y','ts_code_x'],inplace=True)
df07                  

,trade_date,open,high,low,close,pre_close,up_limit,down_limit
0,20200117,6.44,6.44,5.96,6.08,6.62,7.28,5.96
1,20200116,6.10,6.62,6.08,6.62,6.02,6.62,5.42
2,20200115,5.96,6.19,5.87,6.02,5.95,6.55,5.36
3,20200114,6.05,6.10,5.76,5.95,5.92,6.51,5.33
4,20200113,6.00,6.09,5.86,5.92,5.90,6.49,5.31
...,...,...,...,...,...,...,...,...
116,20190730,6.09,6.16,6.06,6.09,6.09,6.70,5.48
117,20190729,6.10,6.13,6.05,6.09,6.10,6.71,5.49
118,20190726,6.07,6.13,6.02,6.10,6.07,6.68,5.46
119,20190725,6.11,6.13,6.04,6.07,6.10,6.71,5.49


In [72]:
ws=df07[(df07.high==df07.up_limit)|(df07.low==df07.down_limit)]
ws

,trade_date,open,high,low,close,pre_close,up_limit,down_limit
0,20200117,6.44,6.44,5.96,6.08,6.62,7.28,5.96
1,20200116,6.10,6.62,6.08,6.62,6.02,6.62,5.42
6,20200109,5.54,6.07,5.54,5.99,5.52,6.07,4.97
32,20191203,4.30,4.74,4.28,4.74,4.31,4.74,3.88
34,20191129,4.29,4.66,4.29,4.43,4.77,5.25,4.29
35,20191128,4.77,4.77,4.77,4.77,5.30,5.83,4.77
36,20191127,5.62,5.78,5.25,5.30,5.25,5.78,4.73
37,20191126,5.11,5.25,5.03,5.25,4.77,5.25,4.29
38,20191125,4.38,4.77,4.33,4.77,4.34,4.77,3.91


In [73]:
ws['flag'] = '最低价=跌停价'
#df.loc[df['name'] == '', 'x4'] = 0
ws.loc[ws.high == ws.up_limit, 'flag'] = '最高价=涨停价'
#ws['flag'] = ws.apply(lambda x: '最高价=涨停价'
#                     if ws.high == ws.up_limit else '最低价=跌停价',
#                      axis=1)

#ws['flag']=np.where(ws.high==ws.up_limit,'最高价=涨停价','最低价=跌停价')
ws.sort_values(by='trade_date')

/Users/bsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/bsg/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,trade_date,open,high,low,close,pre_close,up_limit,down_limit,flag
38,20191125,4.38,4.77,4.33,4.77,4.34,4.77,3.91,最高价=涨停价
37,20191126,5.11,5.25,5.03,5.25,4.77,5.25,4.29,最高价=涨停价
36,20191127,5.62,5.78,5.25,5.30,5.25,5.78,4.73,最高价=涨停价
35,20191128,4.77,4.77,4.77,4.77,5.30,5.83,4.77,最低价=跌停价
34,20191129,4.29,4.66,4.29,4.43,4.77,5.25,4.29,最低价=跌停价
32,20191203,4.30,4.74,4.28,4.74,4.31,4.74,3.88,最高价=涨停价
6,20200109,5.54,6.07,5.54,5.99,5.52,6.07,4.97,最高价=涨停价
1,20200116,6.10,6.62,6.08,6.62,6.02,6.62,5.42,最高价=涨停价
0,20200117,6.44,6.44,5.96,6.08,6.62,7.28,5.96,最低价=跌停价


In [57]:
data = {
    'amount': [100, 200, 300, 400, 500],
    'name': ['', '商品1', '商品3', '', '商品3']
}
test= pd.DataFrame(data)
test

,amount,name
0,100,
1,200,商品1
2,300,商品3
3,400,
4,500,商品3


In [58]:
 test['x2'] = np.where(test['name'] == '', 0, test['amount'])
 test

,amount,name,x2
0,100,,0
1,200,商品1,200
2,300,商品3,300
3,400,,0
4,500,商品3,500


In [52]:
ws.drop(columns='ts_code_x')
ws

,trade_date,open,high,low,close,pre_close,up_limit,down_limit
28,20191209,9.58,10.56,9.50,10.56,9.60,10.56,8.64
38,20191125,10.50,10.51,9.32,9.96,10.36,11.40,9.32
45,20191114,11.06,11.06,10.30,11.06,10.05,11.06,9.05
46,20191113,9.15,10.05,9.15,10.05,9.14,10.05,8.23
67,20191015,10.45,10.45,9.46,9.64,9.50,10.45,8.55
69,20191011,9.00,9.80,8.86,9.60,8.91,9.80,8.02
75,20190926,8.51,8.88,7.73,8.63,8.59,9.45,7.73
76,20190925,8.27,8.90,8.25,8.59,8.09,8.90,7.28
95,20190828,10.40,10.40,9.18,9.18,10.20,11.22,9.18
96,20190827,10.20,10.20,10.20,10.20,9.27,10.20,8.34


,ts_code_x,trade_date,open,high,low,close,pre_close,up_limit,down_limit
28,000007.SZ,20191209,9.58,10.56,9.50,10.56,9.60,10.56,8.64
38,000007.SZ,20191125,10.50,10.51,9.32,9.96,10.36,11.40,9.32
45,000007.SZ,20191114,11.06,11.06,10.30,11.06,10.05,11.06,9.05
46,000007.SZ,20191113,9.15,10.05,9.15,10.05,9.14,10.05,8.23
67,000007.SZ,20191015,10.45,10.45,9.46,9.64,9.50,10.45,8.55
69,000007.SZ,20191011,9.00,9.80,8.86,9.60,8.91,9.80,8.02
75,000007.SZ,20190926,8.51,8.88,7.73,8.63,8.59,9.45,7.73
76,000007.SZ,20190925,8.27,8.90,8.25,8.59,8.09,8.90,7.28
95,000007.SZ,20190828,10.40,10.40,9.18,9.18,10.20,11.22,9.18
96,000007.SZ,20190827,10.20,10.20,10.20,10.20,9.27,10.20,8.34


In [19]:

# 获取时间段统计信息
df07limit_list = pro.limit_list(
    ts_code='603157.SH,000007.SZ', start_date='20190724', end_date='20200118')
df07limit_list.drop(columns='ts_code',inplace=True)
df07limit_list.rename(columns=llCname)

,trade_date,name,close,涨跌幅,振幅,封单额/日成交额,封单手数/流通股本,封单额,首次涨停,最后封板时,打开次数,涨跌停强度,D跌停U涨停
0,20200116,拉夏贝尔,6.62,9.97,8.97,4.5182,1.1541,11139474.0,09:44:36,14:53:37,63,51.539530,U
1,20191209,全新好,10.56,10.00,11.04,11.4832,0.5090,16605601.0,14:50:36,14:52:30,2,50.133770,U
2,20191203,拉夏贝尔,4.74,9.98,10.67,10.9873,1.7947,12403186.0,14:32:15,14:32:51,1,50.518463,U
3,20191128,拉夏贝尔,4.77,-10.00,0.00,237.3472,7.8295,54451688.0,09:25:05,09:25:05,0,73.307150,D
4,20191126,拉夏贝尔,5.25,10.06,4.61,54.8017,3.0743,23532600.0,09:32:17,09:32:17,0,71.023895,U
5,20191125,拉夏贝尔,4.77,9.91,10.14,60.3787,2.1310,14820657.0,09:43:03,09:43:03,0,61.004425,U
6,20191114,全新好,11.06,10.05,7.56,2.1534,0.1978,6760303.5,09:25:03,14:49:24,4,57.665226,U
7,20191113,全新好,10.05,9.96,9.85,11.6980,0.3910,12141405.0,10:25:45,10:33:30,8,51.339348,U
8,20190828,全新好,9.18,-10.00,11.96,1.5870,0.2383,6758775.0,13:20:42,14:22:21,21,48.210617,D
9,20190827,全新好,10.20,10.03,0.00,2400.6067,11.7163,369215008.0,09:25:03,09:25:03,0,100.000000,U
